In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    #input_index = np.array(range(0, length, 24))
    input_index = np.array(range(inputs_day.shape[0]))
    #label_index = input_index + 24* (7 + predict_day)
    label_index = input_index + (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    o_ = np.mean(label_day[:,predict_day,:])
    p_ = np.mean(pred_day[:,predict_day,:])
    
    oi = label_day[:,predict_day,:]
    pi = pred_day[:,predict_day,:]
    
    high = ((oi-o_)*(pi-p_)).sum()
    low = np.sqrt( ( (oi-o_)**2 ).sum() )
    low = low * np.sqrt( ( (pi-p_)**2 ).sum() )
    
    R = high/low
    RS = (R)**2
    
    return nse3, pbias3, pred_day, labels, mae, rmse, RS, R

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e) 


1 Physical GPUs, 1 Logical GPUs


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters['watershed']

'nak_pull'

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 15)

In [12]:
run_num = range(len(folder))
#run_num = [0]
#if file_parameters['watershed'] == 'nak':
#    run_num = range(len(folder))
#else:
#    run_num = [0]

#__GAIN_TRAINING__ = False

real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

ddday = 31
mmmonth = 12

#length = 3

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
                               colum_idx[idx], interpolation=interpolation_option[idx],
                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/nak/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['도개_2016.xlsx', '도개_2017.xlsx', '도개_2018.xlsx'], ['신암_2016.xlsx', '신암_2017.xlsx', '신암_2018.xlsx']]
Epoch 1/2000
1/1 [==============================] - 0s 249ms/step - gen_loss: 100.9799 - disc_loss: 0.7203 - rmse: 0.8986 - val_loss: 1.0102
Epoch 2/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 93.1825 - disc_loss: 0.5314 - rmse: 0.9317 - val_loss: 0.7729
Epoch 3/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 68.6779 - disc_loss: 0.4785 - rmse: 0.7361 - val_loss: 0.7390
Epoch 4/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 64.3612 - disc_loss: 0.4104 - rmse: 0.7143 - val_loss: 0.7363
Epoch 5/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 56.8596 - disc_loss: 0.4078 - rmse: 0.6964 - val_loss: 0.6563
Epoch 6/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 52.9251 - 

1/1 [==============================] - 0s 20ms/step - gen_loss: 7.3307 - disc_loss: 0.3377 - rmse: 0.3294 - val_loss: 0.3842
Epoch 59/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.8361 - disc_loss: 0.3502 - rmse: 0.3211 - val_loss: 0.3318
Epoch 60/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.3205 - disc_loss: 0.3555 - rmse: 0.3688 - val_loss: 0.2791
Epoch 61/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 8.3913 - disc_loss: 0.3278 - rmse: 0.3390 - val_loss: 0.3279
Epoch 62/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 7.0719 - disc_loss: 0.3440 - rmse: 0.3488 - val_loss: 0.3730
Epoch 63/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 7.0109 - disc_loss: 0.3491 - rmse: 0.3297 - val_loss: 0.2889
Epoch 64/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 7.7190 - disc_loss: 0.3222 - rmse: 0.3272 - val_loss: 0.3724
Epoch 65/2000
1/1 [=====================

Epoch 117/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3311 - disc_loss: 0.3096 - rmse: 0.2869 - val_loss: 0.2946
Epoch 118/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.9592 - disc_loss: 0.2829 - rmse: 0.2815 - val_loss: 0.3272
Epoch 119/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.9883 - disc_loss: 0.2823 - rmse: 0.2749 - val_loss: 0.3106
Epoch 120/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1264 - disc_loss: 0.2856 - rmse: 0.2740 - val_loss: 0.2569
Epoch 121/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.1781 - disc_loss: 0.2909 - rmse: 0.2908 - val_loss: 0.2663
Epoch 122/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.9602 - disc_loss: 0.2935 - rmse: 0.2323 - val_loss: 0.2463
Epoch 123/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 6.1769 - disc_loss: 0.2768 - rmse: 0.3174 - val_loss: 0.2893
Epoch 124/2000
1/1 [

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1002 - disc_loss: 0.2887 - rmse: 0.2132 - val_loss: 0.3121
Epoch 176/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7684 - disc_loss: 0.2587 - rmse: 0.2216 - val_loss: 0.3132
Epoch 177/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.4728 - disc_loss: 0.2583 - rmse: 0.2260 - val_loss: 0.2886
Epoch 178/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1052 - disc_loss: 0.2588 - rmse: 0.2101 - val_loss: 0.2850
Epoch 179/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.4181 - disc_loss: 0.2772 - rmse: 0.2328 - val_loss: 0.3081
Epoch 180/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4438 - disc_loss: 0.2713 - rmse: 0.2462 - val_loss: 0.2470
Epoch 181/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.9767 - disc_loss: 0.2664 - rmse: 0.2793 - val_loss: 0.2762
Epoch 182/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.9313 - disc_loss: 0.2556 - rmse: 0.2382 - val_loss: 0.3301
Epoch 234/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0495 - disc_loss: 0.2717 - rmse: 0.2529 - val_loss: 0.3351
Epoch 235/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.3749 - disc_loss: 0.2523 - rmse: 0.2490 - val_loss: 0.2789
Epoch 236/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3825 - disc_loss: 0.2508 - rmse: 0.2196 - val_loss: 0.2531
Epoch 237/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.2517 - disc_loss: 0.2597 - rmse: 0.2509 - val_loss: 0.2123
Epoch 238/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.2065 - disc_loss: 0.2421 - rmse: 0.2205 - val_loss: 0.2758
Epoch 239/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 3.1747 - disc_loss: 0.2422 - rmse: 0.2255 - val_loss: 0.3040
Epoch 240/2000
1/1 [===============

...

1/1 [==============================] - 0s 19ms/step - gen_loss: 0.7605 - disc_loss: 0.0975 - rmse: 0.1183 - val_loss: 0.1550
Epoch 993/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.8619 - disc_loss: 0.1023 - rmse: 0.2563 - val_loss: 0.3170
Epoch 994/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.9566 - disc_loss: 0.0961 - rmse: 0.1599 - val_loss: 0.2482
Epoch 995/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 1.0601 - disc_loss: 0.0946 - rmse: 0.1635 - val_loss: 0.2373
Epoch 996/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 1.0818 - disc_loss: 0.0922 - rmse: 0.1972 - val_loss: 0.1779
Epoch 997/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 0.8587 - disc_loss: 0.0865 - rmse: 0.1318 - val_loss: 0.1299
Epoch 998/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 0.8507 - disc_loss: 0.0964 - rmse: 0.1324 - val_loss: 0.3026
Epoch 999/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 1.0545 - disc_loss: 0.0900 - rmse: 0.3290 - val_loss: 0.2247
Epoch 1051/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 1.1423 - disc_loss: 0.0954 - rmse: 0.2067 - val_loss: 0.2267
Epoch 1052/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 1.0094 - disc_loss: 0.1000 - rmse: 0.2451 - val_loss: 0.1813
Epoch 1053/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.8547 - disc_loss: 0.1006 - rmse: 0.1630 - val_loss: 0.1194
Epoch 1054/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.8989 - disc_loss: 0.0951 - rmse: 0.2749 - val_loss: 0.1672
Epoch 1055/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 0.9148 - disc_loss: 0.0942 - rmse: 0.1809 - val_loss: 0.2294
Epoch 1056/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.8917 - disc_loss: 0.0943 - rmse: 0.1280 - val_loss: 0.1594
Epoch 1057/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 1.4667 - disc_loss: 0.0987 - rmse: 0.1830 - val_loss: 0.1556
Epoch 1109/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.9245 - disc_loss: 0.0921 - rmse: 0.1912 - val_loss: 0.1649
Epoch 1110/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.9356 - disc_loss: 0.0969 - rmse: 0.1860 - val_loss: 0.1404
Epoch 1111/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 0.8655 - disc_loss: 0.0942 - rmse: 0.1614 - val_loss: 0.1180
Epoch 1112/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.8353 - disc_loss: 0.0926 - rmse: 0.1400 - val_loss: 0.2138
Epoch 1113/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.7690 - disc_loss: 0.0907 - rmse: 0.1193 - val_loss: 0.1852
Epoch 1114/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 1.0657 - disc_loss: 0.0899 - rmse: 0.2191 - val_loss: 0.2173
Epoch 1115/2000
1/1 [========

1/1 [==============================] - 0s 19ms/step - gen_loss: 1.0784 - disc_loss: 0.0938 - rmse: 0.1974 - val_loss: 0.1989
Epoch 1166/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.8470 - disc_loss: 0.0887 - rmse: 0.1308 - val_loss: 0.1250
Epoch 1167/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 1.0292 - disc_loss: 0.0940 - rmse: 0.2656 - val_loss: 0.2085
Epoch 1168/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.8512 - disc_loss: 0.1045 - rmse: 0.1681 - val_loss: 0.3304
Epoch 1169/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 1.0139 - disc_loss: 0.0925 - rmse: 0.2059 - val_loss: 0.2426
Epoch 1170/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.8687 - disc_loss: 0.1000 - rmse: 0.1092 - val_loss: 0.1901
Epoch 1171/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 1.3974 - disc_loss: 0.0957 - rmse: 0.1658 - val_loss: 0.1911
Epoch 1172/2000
1/1 [========

row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
interpol flag :  [True, True]
folder :  data/nak/인구/
colum_idx :  :,5:
file_names[idx] :  [['1_2016.xlsx', '1_2017.xlsx', '1_2018.xlsx'], ['2_2016.xlsx', '2_2017.xlsx', '2_2018.xlsx'], ['3_2016.xlsx', '3_2017.xlsx', '3_2018.xlsx'], ['4_2016.xlsx', '4_2017.xlsx', '4_2018.xlsx'], ['5_2016.xlsx', '5_2017.xlsx', '5_2018.xlsx'], ['6_2016.xlsx', '6_2017.xlsx', '6_2018.xlsx'], ['7_2016.xlsx', '7_2017.xlsx', '7_2018.xlsx'], ['8_2016.xlsx', '8_2017.xlsx', '8_2018.xlsx'], ['9_2016.xlsx', '9_2017.xlsx', '9_2018.xlsx'], ['10_2016.xlsx', '10_2017.xlsx', '10_2018.xlsx']]
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
interpol flag :  [True, True]
folder :  data/nak/양식계/
colu

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:

#pd.DataFrame(gan).tail
#real_df_all.tail



In [16]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  2
out_num_features :  1


In [35]:
#ctrl f -- 타겟

print(real_df_all.shape)
rnn_target_column = 'tp'
print(rnn_target_column)



(26304, 1102)
tp


In [36]:

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

train_df, test_df, val_df = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, val=1, test=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (18412, 1102) val_df.shape :  (5261, 1102) test_df.shape: (2631, 1102)


In [37]:
#fake_df.shape[0]/8760

In [38]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)


label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [39]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

In [40]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df)

model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/nak/models/nak_pull/phosphorus/


In [41]:
rnn_epochs, rnn_in_setps, __RNN_TRAINING__

(15, 240, True)

## 모델 학습

In [42]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15, steps_per_epoch = 5,
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=2, steps_per_epoch = 2,
    #training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=False)
    training_flag=True, checkpoint_path=model_path+"gru.ckpt", continue_train=False)

Epoch 1/15
5/5 [==============================] - 134s 27s/step - loss: 0.5524 - mean_absolute_error: 0.5611 - nse: -0.5660 - val_loss: 0.6774 - val_mean_absolute_error: 0.5532 - val_nse: 0.1607
Epoch 2/15
5/5 [==============================] - 123s 25s/step - loss: 0.3793 - mean_absolute_error: 0.4548 - nse: 0.1613 - val_loss: 0.4989 - val_mean_absolute_error: 0.4724 - val_nse: 0.3806
Epoch 3/15
5/5 [==============================] - 122s 24s/step - loss: 0.2554 - mean_absolute_error: 0.3643 - nse: 0.3907 - val_loss: 0.3833 - val_mean_absolute_error: 0.4206 - val_nse: 0.5204
Epoch 4/15
5/5 [==============================] - 122s 24s/step - loss: 0.2036 - mean_absolute_error: 0.3209 - nse: 0.4680 - val_loss: 0.4015 - val_mean_absolute_error: 0.4527 - val_nse: 0.4970
Epoch 5/15
5/5 [==============================] - 122s 24s/step - loss: 0.1833 - mean_absolute_error: 0.2976 - nse: 0.6126 - val_loss: 0.4216 - val_mean_absolute_error: 0.4762 - val_nse: 0.4700
Epoch 6/15
5/5 [=============

In [43]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)
    
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column , " ", out_features[0])
print("length : ",real_df_all.shape[0])
print(teg_check)

print('all_nse : ', val_nse['GRU'])
print('all_pbias : ', val_pbias['GRU'])
print('all_mae : ', _mae)
print('all_rmse : ', _rmse)
print('all_Rs : ', _Rs)
print('all_R : ', _R)



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

save/nak/models/nak_pull/phosphorus/
year : 2016 ~ 2018
run :  range(0, 17)
target :  tp   6
length :  26304
train=7, val=1, test=2
all_nse :  [0.47562008]
all_pbias :  [8.00364183]
all_mae :  0.005832698758271374
all_rmse :  0.016204782546814542
all_Rs :  0.48161545239013887
all_R :  0.6939851960886045


In [44]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=train_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)

print('----------------------------')
print('train_nse : ', val_nse['GRU'])
print('train_pbias : ', val_pbias['GRU'])
print('train_mae : ', _mae)
print('train_rmse : ', _rmse)
print('train_Rs : ', _Rs)
print('train_R : ', _R)

#print('nse_train : ', val_nse['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
train_nse :  [0.88701404]
train_pbias :  [-5.46284268]
train_mae :  0.0037355212643599175
train_rmse :  0.005923027462507449
train_Rs :  0.8920284985966541
train_R :  0.9444726034124304


In [45]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)

print('----------------------------')
print('val_nse : ', val_nse['GRU'])
print('val_pbias : ', val_pbias['GRU'])
print('val_mae : ', _mae)
print('val_rmse : ', _rmse)
print('val_Rs : ', _Rs)
print('val_R : ', _R)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
val_nse :  [-0.58662452]
val_pbias :  [47.63145232]
val_mae :  0.008595563730174342
val_rmse :  0.012448082751208381
val_Rs :  0.2621637332111762
val_R :  0.5120192703513963


In [46]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)

print('----------------------------')
print('test_nse : ', val_nse['GRU'])
print('test_pbias : ', val_pbias['GRU'])
print('test_mae : ', _mae)
print('test_rmse : ', _rmse)
print('test_Rs : ', _Rs)
print('test_R : ', _R)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
test_nse :  [0.50228507]
test_pbias :  [15.16325973]
test_mae :  0.0103016759646712
test_rmse :  0.020413621269796907
test_Rs :  0.5346072658220576
test_R :  0.7311684250718555


In [29]:
#train_df.shape, val_df.shape, test_df.shape

In [30]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvb111nfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [ ]:

sub_flag = True
history_nse = []

bob = 0
b = 1

while sub_flag:
    

    
    gru_model = model_gru(
        #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
        window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15, steps_per_epoch = 5,
        training_flag=True, checkpoint_path=model_path+"gru.ckpt", continue_train=False)
    
    #nse, _, _, _ = compa(
        #model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
        #predict_day = rnn_predict_day)
        #predict_day = 4)
    
    nse, _, _, _, _, _, _, _ = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
    
    if(nse > 0.5):
        sub_flag = False
   
    
    history_nse.append(nse)
    
    print('--------------------------')
    print(history_nse)
    print(nse)
    print('--------------------------')
    
        
    bob += b
    if(bob < -4):
        b = 1
    if(bob > 0):
        b = -1

